Here I am creating zip files that can be uses for exploring the dataset at trace level

In [1]:
import zipfile
from joblib import Parallel, delayed
import os
import tarfile
import pandas as pd
import numpy as np
import time
start = time.time()
to_skip = tuple([2,57,60,64,66,75,98,101,125,129,130,144]) # corrupted files

def process_file(i):
    if to_skip.count(i) > 0:
        return
    
    
    # open file
    file = tarfile.open(f'MSCallGraph_{i}.tar.gz')

    # extracting file
    file.extractall('./')

    file.close()
    df = pd.read_csv(f'MSCallGraph_{i}.csv')
    df=df.drop(['Unnamed: 0','timestamp','rpcid','interface','rt','rpctype'],axis=1)
    df = df.replace('(?)', np.NaN)
    df = df.replace('', np.NaN)
    df = df.replace('NAN', np.NaN)
    df = df.dropna()
    
    filename = f'MSCallGraph_{i}_traces_exploration.csv'
    df.to_csv(filename,index=False)
    
    os.remove(f'MSCallGraph_{i}.csv')
    # Define the file name and the zip file name
    
    zipfilename = f'MSCallGraph_{i}_traces_exploration.zip'

    # Create a ZipFile object and add the file to it
    with zipfile.ZipFile(zipfilename, 'w', compresslevel=9, compression=zipfile.ZIP_LZMA) as zip:
        zip.write(filename)
    os.remove(filename)
# Parallelize the loop using joblib
num_cores = int(os.cpu_count()/4)  # number of cores to use
results = Parallel(n_jobs=num_cores)(delayed(process_file)(i) for i in range(0, 145))


C:\Users\gianl\anaconda3\envs\MasterThesis\lib\site-packages\joblib\externals\loky\process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


In [2]:
duration = time.time() -start
duration

6275.314833402634